In [ ]:
!pip install datasets

## Load Dataset

In [1]:
from datasets import load_dataset, Dataset

In [2]:

ds = load_dataset("mlabonne/FineTome-100k-dedup")['train']

Generating train split:   0%|          | 0/99523 [00:00<?, ? examples/s]

In [6]:
ds = ds.shuffle(seed=42).select(range(10000))  # take a subset for faster processing

In [8]:
ds = ds.map(lambda x: {'conversations':  [ {'role':'user' if msg['from']=='human' else 'assistant' if msg['from']=='gpt' else msg['from'], 'content': msg['value']} for msg in x['conversations']]})  # remove any turn data except role and content

Map:   0%|          | 0/10000 [00:00<?, ? examples/s]

## Anonymize PII Data

In [ ]:
import re

patterns = {
    "EMAIL": r"\b[A-Za-z0-9._%+-]+@[A-Za-z0-9.-]+\.[A-Za-z]{2,}\b",
    "PHONE": r"(?:\+?\d{1,3}[-.\s]?)?(?:\(?\d{2,4}\)?[-.\s]?)?\d{3,4}[-.\s]?\d{4}",
    "CREDIT_CARD": r"\b(?:\d[ -]*?){13,19}\b",
    "SSN": r"\b\d{3}-\d{2}-\d{4}\b",
    "IP": r"\b(?:\d{1,3}\.){3}\d{1,3}\b",
}

anonymized = {
    "EMAIL" : "someone@gmail.com",
    "PHONE": "+90 555 555 5555",
    "CREDIT_CARD": "4111 1111 1111 1111",
    "SSN": "000-00-0000",
    "IP": "192.168.1.0"
}

n_cleaned = 0

def redact_pii(text):
    global n_cleaned
    for label, pattern in patterns.items():
        clean_text = re.sub(pattern, f"<{anonymized[label]}>", text, flags=re.IGNORECASE)
    n_cleaned += (text != clean_text)
    return clean_text

def redact_chat(conversation):
    for con in conversation:
        con['content'] = redact_pii(con['content'])
    return conversation

In [12]:
ds_no_pii = ds.map(lambda x: {'conversations': redact_chat(x['conversations'])})

Map:   0%|          | 0/10000 [00:00<?, ? examples/s]

In [13]:
n_cleaned

6

## Save Clean Data

In [15]:
final_ds= ds_no_pii

In [18]:
final_ds = final_ds.remove_columns(['source', 'score'])

In [19]:
final_ds.push_to_hub("habanoz/FineTome-10k-dedup-clean", private=False, token="xxxx")

Uploading the dataset shards:   0%|          | 0/1 [00:00<?, ?it/s]

Creating parquet from Arrow format:   0%|          | 0/10 [00:00<?, ?ba/s]

Uploading files as a binary IO buffer is not supported by Xet Storage. Falling back to HTTP upload.


CommitInfo(commit_url='https://huggingface.co/datasets/habanoz/FineTome-10k-dedup-clean/commit/55d3724c9f624ac9896aa2cd7b8f528adac921fe', commit_message='Upload dataset', commit_description='', oid='55d3724c9f624ac9896aa2cd7b8f528adac921fe', pr_url=None, repo_url=RepoUrl('https://huggingface.co/datasets/habanoz/FineTome-10k-dedup-clean', endpoint='https://huggingface.co', repo_type='dataset', repo_id='habanoz/FineTome-10k-dedup-clean'), pr_revision=None, pr_num=None)